# CSC 177  Yelp Recommendation System (Simple Recommenders)
# Dr. Haiquan Chen
### Ehsan Hosseinzadeh  Khaligh, Shah Newaz

#### Pre-processing

In [72]:
import pandas as pd 
import time

pd.set_option('display.max_columns', None)  

In [73]:
column_names = ['review_id', 'user_id', 'business_id', 'stars', 'date', 'text', 'useful', 'funny', 'cool','']
df = pd.read_table("dataset/xab.txt", sep=",\"" ,header=None, names=column_names, encoding='ISO-8859-1', dtype='str', engine='python')

In [74]:
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,
0,they had left their whipping towel in my trunk...,"useful"":0","funny"":0","cool"":0}",None,None,None,None,None,None
1,"{""review_id"":""gJtYPlDGfkBOGyZTeV71ug""","user_id"":""T2gJvqmX2wbeam-CUw-fvg""","business_id"":""fQt4D34vcJNtEf8Q4zte3w""","stars"":1","date"":""2016-03-01""","text"":""DISAPPOINTED - I have no clue why every...","useful"":0","funny"":0","cool"":0}",None
2,"{""review_id"":""VbbXwdwY41kd-uBNy6_3Xg""","user_id"":""T2gJvqmX2wbeam-CUw-fvg""","business_id"":""e0JOkQYz_cnz91k6X55PLw""","stars"":5","date"":""2016-03-01""","text"":""In line with the first class experience...","useful"":0","funny"":0","cool"":0}",None
3,"{""review_id"":""AezK5-RvRp-VGGnrI3Om-A""","user_id"":""-EArvT0PKftzn4QYVqJKfg""","business_id"":""Gaq3S9lmjXVcuDCZ8ulppw""","stars"":5","date"":""2017-08-26""","text"":""We at the all you can eat AYCE and it w...","useful"":0","funny"":0","cool"":0}",None
4,"{""review_id"":""BPaWfPnL3e8mJQ0mJmcjkg""","user_id"":""JDu-gXhyLCHohpzoVWbmqA""","business_id"":""XEvOc1jhPosMn69hoqiYPQ""","stars"":3","date"":""2015-07-05""","text"":""Great salon but a bit pricy!!! I pay $8...","useful"":0","funny"":0","cool"":0}",None


In [75]:
filterd_df = df[["user_id","business_id", "stars", "useful"]]
filterd_df = filterd_df.drop(filterd_df.index[0])

In [76]:
filterd_df["user_id"] = filterd_df["user_id"].apply((lambda string: string.replace("user_id\":\"", "")))
filterd_df["user_id"] = filterd_df["user_id"].apply((lambda string: string.replace("\"", "")))
filterd_df["business_id"] = filterd_df["business_id"].apply((lambda string: string.replace("business_id\":\"", "")))
filterd_df["business_id"] = filterd_df["business_id"].apply((lambda string: string.replace("\"", "")))
filterd_df["stars"] = filterd_df["stars"].apply((lambda string: string.replace("stars", "")))
filterd_df["stars"] = filterd_df["stars"].apply((lambda string: string.replace(":", "")))
filterd_df["stars"] = filterd_df["stars"].apply((lambda string: string.replace("\"", "")))
filterd_df["useful"] = filterd_df["useful"].apply((lambda string: string.replace("useful", "")))
filterd_df["useful"] = filterd_df["useful"].apply((lambda string: string.replace(":", "")))
filterd_df["useful"] = filterd_df["useful"].apply((lambda string: string.replace("\"", "")))

In [77]:
filterd_df = filterd_df[filterd_df.useful !=  "None"]

In [78]:
filterd_df.head()

,user_id,business_id,stars,useful
1,T2gJvqmX2wbeam-CUw-fvg,fQt4D34vcJNtEf8Q4zte3w,1,0
2,T2gJvqmX2wbeam-CUw-fvg,e0JOkQYz_cnz91k6X55PLw,5,0
3,-EArvT0PKftzn4QYVqJKfg,Gaq3S9lmjXVcuDCZ8ulppw,5,0
4,JDu-gXhyLCHohpzoVWbmqA,XEvOc1jhPosMn69hoqiYPQ,3,0
5,bNg6dGsyx5SB4rO5kU8zmQ,fuW-VCynECpKukrm-9nxdg,1,1


- Calculate 'stars' avarage for all the users

In [79]:
mean = filterd_df.stars.astype(float).mean()
print(mean)

3.727611778216898


- find  ’useful’ rate, received by the user in the 90th percentile

In [80]:
percentile = filterd_df.useful.astype(float).quantile(0.90)
print(percentile)

4.0


#### Applying simple recommender

- Copy data into new dataframe

In [81]:
recRest = filterd_df.copy().loc[filterd_df.useful.astype(float) >= percentile]
recRest.head(10)

,user_id,business_id,stars,useful
6,oPawJCZrdkLO2PlhVcdfxQ,Bu00AQVhokwJESYxqInPKw,1,4
13,qwURNQ11dEFTKI3Dn3Mb2A,X8c23dur0ll2D9XTu-I8Qg,2,10
16,qwURNQ11dEFTKI3Dn3Mb2A,7sb2FYLS2sejZKxRYF9mtg,5,12
17,qwURNQ11dEFTKI3Dn3Mb2A,3_QNAH8yVzY0sBPftWyNhQ,1,14
45,KAdPUgHNHtR0n1OMBxSIMw,o5nPu-ITfZ2OYx-6cU3moQ,5,4
64,c33ze-_RrQh5iIv_M2Ql3g,19pozupnPYEtdWTzUSatBQ,4,4
66,c33ze-_RrQh5iIv_M2Ql3g,UtWngqS-WloIY_A53W5K-Q,5,7
84,Odf5wj9E-a48fBTKdPPxPw,E3EVdQrzmLGa8dls1-DhIQ,1,4
93,UnyJSl62ZWROxk9NaUWKPQ,16jQV_PRCSRTSwdTDSHxwg,1,4
102,_yvsRfxeGGbjeNTV5m7YkQ,SgG5E7PUrCIsEMP0BVSlTg,5,16


Weighted Rating Equation:

WR = ( V /  ( V + M)  * R) + ( M / ( M + V ) * C ) 

Where:

V is the ‘stars’ value for each record 

M is  the ’useful’ rate, received by the user in the 90th 
percentile


R is the ‘useful’ value for each record 

C is  the mean/average rate of ‘stars’ 


In [82]:
#https://www.datacamp.com/community/tutorials/recommender-systems-python
def weighted_rating(x, m=percentile, C=mean):
    v = float(x.stars)
    R = float(x.useful)
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [83]:
recRest['score'] = recRest.apply(weighted_rating, axis=1)

In [84]:
recRest = recRest.sort_values('score', ascending=False)
recRest.head(15)

,user_id,business_id,stars,useful,score
349842,T8LqkYbwMPZmvXSsUfN8sQ,DN0b4Un8--Uf6SEWLeh0UA,2,1608,538.485075
453494,TG3GcRx20tAxXxpqn20Lwg,DN0b4Un8--Uf6SEWLeh0UA,1,782,159.382089
1312049,2SW0GXcPc2eTfykYfZNeBQ,DN0b4Un8--Uf6SEWLeh0UA,3,357,155.130064
912876,63RMC0LxibWFTlHOGgseGw,DN0b4Un8--Uf6SEWLeh0UA,1,679,138.782089
498958,2gmKNhmQrS5bGo518WWzcw,DN0b4Un8--Uf6SEWLeh0UA,1,647,132.382089
596978,ImqnI6Q1qNjxjwP4AE6RhQ,DN0b4Un8--Uf6SEWLeh0UA,1,627,128.382089
673240,--2vR0DIsmQ6WfcSzKWigw,7dHYudt6OOIjiaxkSvv3lQ,5,203,114.434494
673239,--2vR0DIsmQ6WfcSzKWigw,Ka00H3EHLLiPNpMj1V4Hcw,5,202,113.878939
673234,--2vR0DIsmQ6WfcSzKWigw,t-o_Sraneime4DDhWrQRBA,5,198,111.656716
673263,--2vR0DIsmQ6WfcSzKWigw,g-32bI6DCI0DJh0dD5Y-aQ,5,190,107.212272


In [64]:
bussList = recRest.business_id.tolist()
bussList

['DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 '7dHYudt6OOIjiaxkSvv3lQ',
 'Ka00H3EHLLiPNpMj1V4Hcw',
 't-o_Sraneime4DDhWrQRBA',
 'g-32bI6DCI0DJh0dD5Y-aQ',
 '3xmfT7l3xNH5LK1dLzfvGw',
 '9edPSkfXKsJmkZYIaOmA7Q',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'gm8BAH6eZrDkJv0CCas0Bg',
 'u02xse4VO7lkyF4r7ARNdw',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'sARavfidmIS878q5BtlyVQ',
 '7U-yh8dvsvhd87i2crfOAw',
 'JPfi__QJAaRzmfh5aOyFEw',
 '7l8IkCAwZ3e2jPY2faZSgQ',
 'vOMDU31gdylrzBhAKC9QbA',
 'k1TXUt_RxdIQdoOoxLPR-g',
 'yFFFiuPQdHr-KPqD9s2LsQ',
 'IhNASEZ3XnBHmuuVnWdIwA',
 'beuVp5CZxCdNvQIIPBS2rw',
 'Wxxvi3LZbHNIDwJ-ZimtnA',
 'pyqnGlIfP9Zw8LLxBjyAiA',
 'na4Th5DrNauOv-c43QQFvA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'DN0b4Un8--Uf6SEWLeh0UA',
 'SUH0cMrRsysIu2hKqQQKBA',
 'bAa0ue-BJychrcJZd-xSzw',
 'DxgSVufr_6y_Rc63CC17Zg',
 'uh7DUWtPoZkuEE05fghJ_w',
 'Y07qIurPKiUJDnTZWxZN2A',
 'DfgZlNgKwBvCpA_0alumXw',
 

#### Top 5 Recommened Restaurants:

- Look at orginal bussiness JSON file to find the restaurant. 

First Restaurant/Business:

In [ ]:
{
	"business_id": "DN0b4Un8--Uf6SEWLeh0UA",
	"name": "Amy's Baking Company",
	"neighborhood": "",
	"address": "7366 E Shea Blvd, Ste 112",
	"city": "Scottsdale",
	"state": "AZ",
	"postal_code": "85260",
	"latitude": 33.584012,
	"longitude": -111.922265,
	"stars": 3.0,
	"review_count": 461,
	"is_open": 0,
	"attributes": {
		"GoodForMeal": {
			"dessert": true,
			"latenight": false,
			"lunch": false,
			"dinner": false,
			"breakfast": false,
			"brunch": false
		},
		"Alcohol": "full_bar",
		"Caters": false,
		"BusinessAcceptsCreditCards": true,
		"RestaurantsPriceRange2": 2,
		"BikeParking": false,
		"RestaurantsDelivery": false,
		"RestaurantsTakeOut": true,
		"ByAppointmentOnly": false,
		"WheelchairAccessible": true,
		"BusinessParking": {
			"garage": false,
			"street": false,
			"validated": false,
			"lot": true,
			"valet": false
		}
	},
	"categories": ["American (New)", "Pizza", "Bakeries", "Italian", "Food", "Restaurants"],
	"hours": {
		"Tuesday": "12:00-21:00",
		"Friday": "12:00-21:00",
		"Wednesday": "12:00-21:00",
		"Thursday": "12:00-21:00",
		"Sunday": "12:00-21:00",
		"Saturday": "12:00-21:00"
	}
}

Second Restaurant/Business:

In [ ]:
{
	"business_id": "7dHYudt6OOIjiaxkSvv3lQ",
	"name": "In-N-Out Burger",
	"neighborhood": "Westside",
	"address": "2900 W Sahara Ave",
	"city": "Las Vegas",
	"state": "NV",
	"postal_code": "89102",
	"latitude": 36.1448187,
	"longitude": -115.1805851,
	"stars": 4.0,
	"review_count": 361,
	"is_open": 1,
	"attributes": {
		"Alcohol": "none",
		"HasTV": false,
		"NoiseLevel": "average",
		"RestaurantsAttire": "casual",
		"BusinessAcceptsCreditCards": true,
		"Ambience": {
			"romantic": false,
			"intimate": false,
			"classy": false,
			"hipster": false,
			"divey": false,
			"touristy": false,
			"trendy": false,
			"upscale": false,
			"casual": true
		},
		"RestaurantsGoodForGroups": true,
		"BYOBCorkage": "yes_free",
		"Caters": false,
		"WiFi": "no",
		"RestaurantsReservations": false,
		"BYOB": false,
		"RestaurantsTableService": false,
		"Corkage": false,
		"RestaurantsTakeOut": true,
		"GoodForKids": true,
		"DriveThru": true,
		"BikeParking": true,
		"OutdoorSeating": true,
		"RestaurantsPriceRange2": 1,
		"RestaurantsDelivery": false,
		"GoodForMeal": {
			"dessert": false,
			"latenight": true,
			"lunch": true,
			"dinner": true,
			"breakfast": false,
			"brunch": false
		},
		"BusinessParking": {
			"garage": false,
			"street": false,
			"validated": false,
			"lot": true,
			"valet": false
		},
		"WheelchairAccessible": true,
		"RestaurantsCounterService": true
	},
	"categories": ["Burgers", "Fast Food", "Restaurants"],
	"hours": {
		"Monday": "10:30-1:00",
		"Tuesday": "10:30-1:00",
		"Friday": "10:30-1:30",
		"Wednesday": "10:30-1:00",
		"Thursday": "10:30-1:00",
		"Sunday": "10:30-1:00",
		"Saturday": "10:30-1:30"
	}
}

Third Restaurant/Business:

In [ ]:
{
	"business_id": "g-32bI6DCI0DJh0dD5Y-aQ",
	"name": "99 Cents Only Stores",
	"neighborhood": "University",
	"address": "1325 E Flamingo Rd",
	"city": "Las Vegas",
	"state": "NV",
	"postal_code": "89119",
	"latitude": 36.114016,
	"longitude": -115.136287,
	"stars": 3.5,
	"review_count": 24,
	"is_open": 1,
	"attributes": {
		"BusinessAcceptsCreditCards": true,
		"RestaurantsPriceRange2": 1,
		"BusinessParking": {
			"garage": false,
			"street": false,
			"validated": false,
			"lot": true,
			"valet": false
		},
		"BikeParking": true,
		"WheelchairAccessible": true
	},
	"categories": ["Fashion", "Party Supplies", "Event Planning & Services", "Discount Store", "Shopping", "Department Stores", "Wholesalers", "Professional Services"],
	"hours": {
		"Monday": "8:00-21:00",
		"Tuesday": "8:00-21:00",
		"Friday": "8:00-21:00",
		"Wednesday": "8:00-21:00",
		"Thursday": "8:00-21:00",
		"Sunday": "8:00-21:00",
		"Saturday": "8:00-21:00"
	}
}

Fourth Restaurant/Business:

In [ ]:
{
	"business_id": "3xmfT7l3xNH5LK1dLzfvGw",
	"name": "Fremont Street Experience",
	"neighborhood": "Downtown",
	"address": "425 Fremont St",
	"city": "Las Vegas",
	"state": "NV",
	"postal_code": "89101",
	"latitude": 36.169182,
	"longitude": -115.141881,
	"stars": 3.5,
	"review_count": 1311,
	"is_open": 1,
	"attributes": {
		"WheelchairAccessible": true,
		"DogsAllowed": false,
		"BusinessParking": {
			"garage": true,
			"street": false,
			"validated": false,
			"lot": false,
			"valet": false
		},
		"HasTV": true,
		"RestaurantsGoodForGroups": true,
		"CoatCheck": false,
		"WiFi": "no",
		"NoiseLevel": "very_loud",
		"OutdoorSeating": true,
		"BusinessAcceptsCreditCards": true,
		"RestaurantsPriceRange2": 1,
		"Music": {
			"dj": false,
			"background_music": false,
			"no_music": false,
			"karaoke": false,
			"live": true,
			"video": false,
			"jukebox": false
		},
		"RestaurantsReservations": false,
		"BikeParking": true,
		"Smoking": "yes",
		"Ambience": {
			"romantic": false,
			"intimate": false,
			"classy": false,
			"hipster": false,
			"divey": false,
			"touristy": true,
			"trendy": false,
			"upscale": false,
			"casual": false
		},
		"BestNights": {
			"monday": false,
			"tuesday": false,
			"friday": true,
			"wednesday": false,
			"thursday": false,
			"sunday": true,
			"saturday": true
		},
		"GoodForKids": true,
		"HappyHour": true,
		"GoodForDancing": true,
		"Alcohol": "full_bar"
	},
	"categories": ["Local Flavor", "Casinos", "Bars", "Nightlife", "Arts & Entertainment"],
	"hours": {}
}

Fifth Restaurant/Business:

In [ ]:
{
	"business_id": "gm8BAH6eZrDkJv0CCas0Bg",
	"name": "F Pigalle",
	"neighborhood": "Downtown",
	"address": "508 E Fremont St",
	"city": "Las Vegas",
	"state": "NV",
	"postal_code": "89101",
	"latitude": 36.169109022,
	"longitude": -115.140062459,
	"stars": 4.0,
	"review_count": 86,
	"is_open": 0,
	"attributes": {
		"Alcohol": "full_bar",
		"HasTV": false,
		"NoiseLevel": "loud",
		"RestaurantsAttire": "casual",
		"BusinessAcceptsCreditCards": true,
		"Music": {
			"dj": false,
			"background_music": true,
			"no_music": false,
			"karaoke": false,
			"live": false,
			"video": false,
			"jukebox": false
		},
		"Ambience": {
			"romantic": false,
			"intimate": false,
			"classy": false,
			"hipster": true,
			"divey": false,
			"touristy": false,
			"trendy": false,
			"upscale": false,
			"casual": false
		},
		"RestaurantsGoodForGroups": true,
		"WiFi": "no",
		"RestaurantsReservations": true,
		"RestaurantsTableService": true,
		"Smoking": "outdoor",
		"GoodForKids": false,
		"HappyHour": true,
		"GoodForDancing": false,
		"BikeParking": true,
		"OutdoorSeating": true,
		"RestaurantsPriceRange2": 2,
		"RestaurantsDelivery": false,
		"GoodForMeal": {
			"dessert": false,
			"latenight": false,
			"lunch": false,
			"dinner": true,
			"breakfast": false,
			"brunch": false
		},
		"BusinessParking": {
			"garage": false,
			"street": true,
			"validated": false,
			"lot": false,
			"valet": false
		},
		"CoatCheck": false,
		"RestaurantsTakeOut": false
	},
	"categories": ["Fondue", "Restaurants", "Lounges", "Cocktail Bars", "Bars", "Nightlife"],
	"hours": {
		"Tuesday": "16:00-4:00",
		"Friday": "16:00-4:00",
		"Wednesday": "16:00-4:00",
		"Thursday": "16:00-4:00",
		"Sunday": "16:00-4:00",
		"Saturday": "16:00-4:00"
	}
}